In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


In [3]:
 # Load the CSV file with feature data
features_df = pd.read_csv('gender_classification_v7.csv')

# Separate features and target from the CSV file
X_features = features_df[['long_hair', 'forehead_width_cm', 'forehead_height_cm']]
y_features = features_df['gender']

# Standardize the feature data
scaler = StandardScaler()
X_features_scaled = scaler.fit_transform(X_features)

# Prepare ImageDataGenerator for loading images
data_gen = ImageDataGenerator(rescale=1.0/255.0)
image_data = data_gen.flow_from_directory(
    'Training', 
    target_size=(150, 150),
    class_mode='binary',  # binary for male/female classification
    batch_size=32,
    shuffle=True
)


Found 42202 images belonging to 2 classes.


In [4]:
# Base model for image feature extraction (pre-trained InceptionV3)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
image_predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# Final model combining both image and feature data
image_model = Model(inputs=base_model.input, outputs=image_predictions)
image_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
# Train the model using the images in the "Training" folder
image_model.fit(image_data, epochs=0)


In [9]:
def predict_gender(filepath, age, long_hair, forehead_width, forehead_height):
    # Determine the gender based on the custom age rule
    if 20 <= age <= 30:
        # Override gender based on hair length
        predicted_gender = 'Female' if long_hair == 1 else 'Male'
    else:
        # Use the feature-based model for individuals outside 20-30 age range
        features = scaler.transform([[long_hair, forehead_width, forehead_height]])
        predicted_gender = model.predict(features)[0]

    return predicted_gender


In [8]:
def upload_image():
    # Open file dialog to select an image file
    filepath = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if filepath:
        # Display selected image
        img = Image.open(filepath)
        img = img.resize((200, 200), Image.LANCZOS)  # Resize with LANCZOS filter for quality
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img

        # Simulate retrieving metadata for this example
        age = 25  # Assuming age for testing
        long_hair = 1  # Assuming long hair for testing
        forehead_width = 14.0  # Example width
        forehead_height = 7.0  # Example height

        # Perform gender prediction based on rules
        predicted_gender = predict_gender(filepath, age, long_hair, forehead_width, forehead_height)
        result_text = f"Predicted Gender: {predicted_gender}\nAge: {age}"
        output_label.config(text=result_text)

# Set up Tkinter GUI
root = tk.Tk()
root.title("Long Hair Gender Prediction")
root.geometry("400x500")

# Button to upload image
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack()

# Image label for displaying uploaded image
image_label = tk.Label(root)
image_label.pack()

# Define the output label here to display prediction results
output_label = tk.Label(root, text="Results will be shown here", wraplength=300, justify="left")
output_label.pack()

# Run the GUI main loop
root.mainloop()
